In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import shutil
import sagemaker
import requests as req
from pathlib import Path 
from sagemaker.s3 import S3Uploader
from sagemaker import get_execution_role
from huggingface_hub import snapshot_download
from sagemaker.huggingface.model import HuggingFaceModel


In [ ]:
# global constants
!pygmentize globals.py

In [ ]:
%run -i globals.py
%run -i utils.py

In [ ]:
create_s3_structure()

In [ ]:
model_dir = HF_MODEL_ID.split("/")[-1]
model_dir

In [ ]:
model_tar_gz_path = os.path.join(os.path.dirname(os.getcwd()), f"model_{model_dir}.tar.gz")
model_tar_gz_path

In [ ]:
%%time
model_path = os.path.join(os.path.dirname(os.getcwd()), model_dir)
Path(model_path).mkdir(exist_ok=True)
# Download model from Hugging Face into model_dir
snapshot_download(HF_MODEL_ID, local_dir=model_path, local_dir_use_symlinks=False)

In [ ]:
inf_dest = os.path.join(model_path, 'code', 'inference.py')
shutil.copyfile("llava_inference.py", inf_dest)

In [ ]:
%%time
# Create SageMaker model.tar.gz artifact
!cd {model_path};tar -cf {model_tar_gz_path} --use-compress-program=pigz *;cd -

In [ ]:
%%time
# upload model.tar.gz to s3
S3Uploader.upload(local_path=model_tar_gz_path, desired_s3_uri=S3_MODEL_URI)

print(f"model uploaded to: {S3_MODEL_URI}")

In [ ]:
%%time

role = get_execution_role()
print(role)

hub = {
  'HF_TASK':HF_TASK
}
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=S3_MODEL_URI,                    # path to your model and script
   role=role,                                  # iam role with permissions to create an Endpoint
   transformers_version=TRANSFORMERS_VERSION,  # transformers version used
   pytorch_version=PYTORCH_VERSION,            # pytorch version used
   py_version=PYTHON_VERSION,                  # python version used
   model_server_workers=1,
   env=hub
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
)

In [ ]:
with open("./endpointname.txt", 'w') as endpoint:
    endpoint.write(predictor.endpoint_name)